<a href="https://colab.research.google.com/github/usshaa/BK_Birla/blob/main/04-Unsupervised/Recommendation_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1️⃣ Content-Based Filtering (Using TF-IDF & Cosine Similarity)**
This method recommends items **based on content similarity**.  

### **Example Scenario**:  
We have a **Movie Dataset** with genres, and we recommend similar movies based on genres.  

### **Step 1: Create Example Dataset**
```python


In [2]:
import pandas as pd

# Sample movie dataset
movies = pd.DataFrame({
    'Movie_ID': [1, 2, 3, 4, 5],
    'Title': ['Inception', 'Interstellar', 'Titanic', 'Avatar', 'The Matrix'],
    'Genres': ['Sci-Fi, Action', 'Sci-Fi, Drama', 'Romance, Drama', 'Sci-Fi, Adventure', 'Sci-Fi, Action']
})

print(movies)

   Movie_ID         Title             Genres
0         1     Inception     Sci-Fi, Action
1         2  Interstellar      Sci-Fi, Drama
2         3       Titanic     Romance, Drama
3         4        Avatar  Sci-Fi, Adventure
4         5    The Matrix     Sci-Fi, Action


### **Step 2: Convert Genres into TF-IDF Vectors**
We use **TfidfVectorizer** to transform genres into numerical values.  


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Convert genres into numerical vectors
vectorizer = TfidfVectorizer(stop_words='english')
genre_matrix = vectorizer.fit_transform(movies['Genres'])

# Compute Cosine Similarity
cosine_sim = cosine_similarity(genre_matrix)

# Display similarity matrix
print(pd.DataFrame(cosine_sim, index=movies['Title'], columns=movies['Title']))


Title         Inception  Interstellar   Titanic    Avatar  The Matrix
Title                                                                
Inception      1.000000      0.493733  0.000000  0.437857    1.000000
Interstellar   0.493733      1.000000  0.446776  0.437857    0.493733
Titanic        0.000000      0.446776  1.000000  0.000000    0.000000
Avatar         0.437857      0.437857  0.000000  1.000000    0.437857
The Matrix     1.000000      0.493733  0.000000  0.437857    1.000000


### **Step 3: Recommend Similar Movies**
Now, we recommend movies similar to **"Inception"**.

In [11]:
def recommend_movie(movie_title, movies, cosine_sim):
    idx = movies[movies['Title'] == movie_title].index[0]  # Get index of the movie
    sim_scores = list(enumerate(cosine_sim[idx]))  # Get similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)  # Sort by similarity
    sim_scores = sim_scores[1:3]  # Top 2 similar movies (excluding itself)

    movie_indices = [i[0] for i in sim_scores]
    return movies['Title'].iloc[movie_indices]

# Recommend similar movies to "Inception"
print(recommend_movie('Inception', movies, cosine_sim))

4      The Matrix
1    Interstellar
Name: Title, dtype: object


## **2️⃣ Collaborative Filtering (Using User-Item Matrix & Pearson Similarity)**
This method **recommends items based on user preferences**.

### **Example Scenario**:  
We have **user ratings for movies**.  

---

### **Step 1: Create Example User Ratings Dataset**
```python

In [5]:
ratings = pd.DataFrame({
    'User': ['Alice', 'Alice', 'Bob', 'Bob', 'Charlie', 'Charlie'],
    'Movie': ['Inception', 'Titanic', 'Inception', 'Avatar', 'Titanic', 'Interstellar'],
    'Rating': [5, 3, 4, 2, 4, 5]
})

print(ratings)

      User         Movie  Rating
0    Alice     Inception       5
1    Alice       Titanic       3
2      Bob     Inception       4
3      Bob        Avatar       2
4  Charlie       Titanic       4
5  Charlie  Interstellar       5


### **Step 2: Create User-Item Matrix**
```python


In [6]:
user_movie_matrix = ratings.pivot(index='User', columns='Movie', values='Rating').fillna(0)
print(user_movie_matrix)


Movie    Avatar  Inception  Interstellar  Titanic
User                                             
Alice       0.0        5.0           0.0      3.0
Bob         2.0        4.0           0.0      0.0
Charlie     0.0        0.0           5.0      4.0


### **Step 3: Compute Similarity Between Users**

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

user_sim = cosine_similarity(user_movie_matrix)
user_sim_df = pd.DataFrame(user_sim, index=user_movie_matrix.index, columns=user_movie_matrix.index)

print(user_sim_df)


User        Alice       Bob   Charlie
User                                 
Alice    1.000000  0.766965  0.321403
Bob      0.766965  1.000000  0.000000
Charlie  0.321403  0.000000  1.000000


### **Step 4: Recommend Movies for a User**

In [12]:
def recommend_movies(user, user_sim_df, ratings, top_n=2):
    similar_users = user_sim_df[user].sort_values(ascending=False).index[1:]  # Exclude itself
    recommended_movies = ratings[ratings['User'].isin(similar_users)]['Movie'].unique()
    return recommended_movies[:top_n]

# Recommend movies for Alice
print(recommend_movies('Alice', user_sim_df, ratings))


['Inception' 'Avatar']


## **3️⃣ Hybrid Recommender System (Combining Content & Collaborative Filtering)**
This method **combines both techniques** to improve accuracy.

---

### **Step 1: Combine Content-Based & Collaborative Filtering Results**

In [15]:
# Get content-based recommendations
content_recommendations = recommend_movie('Inception', movies, cosine_sim)

# Get collaborative filtering recommendations
collab_recommendations = recommend_movies('Alice', user_sim_df, ratings)

# Combine both recommendations
hybrid_recommendations = list(set(content_recommendations) | set(collab_recommendations))

print("Hybrid Recommendations for Alice:", hybrid_recommendations)

Hybrid Recommendations for Alice: ['Interstellar', 'The Matrix', 'Inception', 'Avatar']


## **🎯 Model Comparision**
| **Model Type**         | **Approach**                         | **Python Technique Used**  |
|------------------------|-------------------------------------|---------------------------|
| **Content-Based**      | Find similar items (e.g., movies) based on attributes | TF-IDF + Cosine Similarity |
| **Collaborative**      | Find similar users and recommend items they like | User-Item Matrix + Cosine Similarity |
| **Hybrid**            | Combine both Content & Collaborative Filtering | Merging both techniques |